#### MST AIDS 2023-2024 (Département Génie Informatique)
#### Subject : The main purpose behind this lab is to get familiar with NLP language models using Pytorch library.
#### Realize by : Chibani Fahd
#### web sourcest : Aljazeera
#### Topic : " الحرب الفلسطينية "
#### Models : RNN, Bidirectional RNN, GRU and LSTM

## Part I : Classification Regression

In [46]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
import numpy as np
import json
import pyarabic.araby as araby
from tashaphyne.stemming import ArabicLightStemmer
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import qalsadi.lemmatizer as lem
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, GRU, Dense
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping

### 1 - Scrapping using Selenium and BeautifulSoup :

a )  Scarpe all links in relation with " الحرب الفلسطينية" from aljazeera web site

In [ ]:

# Configure options for Chrome
options = Options()
options.add_argument('--headless')  # Run in headless mode for non-GUI environments
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')

# Set up the Chrome WebDriver
driver = webdriver.Chrome()


# Open the website
driver.get('https://www.aljazeera.net/where/arab/palestine/')

links = set()
for i in range(30):
    # Wait until the links are present
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'u-clickable-card__link'))
    )

    # Find all link elements
    link_elements = driver.find_elements(By.CLASS_NAME, 'u-clickable-card__link')
    for element in link_elements:
        links.add(element.get_attribute('href'))

    # Try to find and click the "Show More" button
    show_more_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/div[3]/div/div[3]/div/div[1]/section/button'))
    )
    driver.execute_script('arguments[0].scrollIntoView();', show_more_button)
    show_more_button.click()
    time.sleep(3)  # Wait for new content to load

# Output all links
for link in links:
    print(link)

driver.quit()


b ) Using BeautifulSoup to scrape informations already scraped :

In [ ]:
para=[]
for url in links:
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    para_div = soup.find('div', {'class': 'wysiwyg wysiwyg--all-content css-1vkfgk0'})
    if para_div:
        print("URL:", url)


        # Initialize a variable to store all paragraphs
        all_paragraphs = ""

        # Extract paragraphs
        for item in para_div.contents:
            if item.name == 'p':
                all_paragraphs += item.text.strip() + "  "

        print("Paragraphs:", all_paragraphs)
        para.append(all_paragraphs)
        print("\n")  # Add a new line for separation between articles
        # Insert data into MongoDB
        article_data = {
            "url": url,
            "paragraphs": all_paragraphs
        }


c ) Save informations in a json file named para.json :

In [ ]:
# Step 3: Open (or create) the 'para.json' file in write mode
with open('para.json', 'w') as json_file:
    # Step 4: Use json.dump() to write the list to the file
    json.dump(para, json_file)


d ) Collecting other information so we can get varies ratings :

In [ ]:
# Configure options for Chrome
options = Options()
options.add_argument('--headless')  # Run in headless mode for non-GUI environments
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')

# Set up the Chrome WebDriver
driver = webdriver.Chrome()


# Open the website
driver.get('https://www.aljazeera.net/')

links = set()
for i in range(1):
    # Wait until the links are present
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'u-clickable-card__link'))
    )

    # Find all link elements
    link_elements = driver.find_elements(By.CLASS_NAME, 'u-clickable-card__link')
    for element in link_elements:
        links.add(element.get_attribute('href'))

# Output all links
for link in links:
    print(link)

driver.quit()

In [ ]:
para=[]
for url in links:
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    para_div = soup.find('div', {'class': 'wysiwyg wysiwyg--all-content css-1vkfgk0'})
    if para_div:
        print("URL:", url)


        # Initialize a variable to store all paragraphs
        all_paragraphs = ""

        # Extract paragraphs
        for item in para_div.contents:
            if item.name == 'p':
                all_paragraphs += item.text.strip() + "  "

        print("Paragraphs:", all_paragraphs)
        para.append(all_paragraphs)
        print("\n")  # Add a new line for separation between articles
        # Insert data into MongoDB
        article_data = {
            "url": url,
            "paragraphs": all_paragraphs
        }

In [ ]:
# Create a list to hold the dictionaries
data_list = []

# Iterate over each item in the 'para' list
for content in para:
    # Create a dictionary with 'content' and 'title'
    data = {
        "title": "None",
        "content": content,
    }

    # Add the dictionary to the list
    data_list.append(data)

# Write the list of dictionaries to a single JSON file
with open('para2.json', 'w', encoding='utf-8') as json_file:
    json.dump(data_list, json_file, ensure_ascii=False, indent=4)


### 2 ) Preprocessing of the collected Dataset.

In [3]:
# Read the JSON file with lines=True
df = pd.read_json("filtered_&_scored_data.json", lines=True)

# Drop the column named 'B'
if 'B' in df.columns:
    df = df.drop(columns=['B'])

# Save the updated DataFrame back to a JSON file
df.to_json('updated_para.json', orient='records', lines=True, force_ascii=False, indent=4)

# Print the updated DataFrame
print(df)


                                               content    rating
0    في هذا المحتوى، سنقدم نظرة عامة على الثقافة ال...  2.398908
1    في هذا المحتوى، سنقدم نظرة عامة على الثقافة ال...  1.097328
2    يستكشف هذا المحتوى دور الذكاء الاصطناعي في مست...  3.398908
3    يتناول هذا المحتوى تحليلًا لتطور صناعة الألعاب...  0.863688
4    يقدم هذا المحتوى استعراضًا لتأثير التغيرات الد...  2.398908
..                                                 ...       ...
223  حاول الغرب، على مدى السنوات القليلة الماضية، ك...  1.148186
224  يحذر خبراء في جنوب أفريقيا من عودة الرئيس السا...  1.615157
225  في سياق إقليمي معقد تمر به منطقة أفريقيا جنوب ...  3.324926
226  قرر يوفنتوس الإيطالي الطعن في حكم كريستيانو رو...  0.855067
227  في هذا التحليل، سنستكشف أحدث التطورات في مجال ...  2.425226

[228 rows x 2 columns]


In [4]:
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى",'\n', '\t','"','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    # remove HTML TAG
    html = re.compile('[<،,()"".#*?>]')
    text = html.sub(r'',text)

    # Remove urls:
    url = re.compile('https?://\S+|www\.S+')
    text = url.sub(r'',text)

    # Remove email id:
    email = re.compile('[A-Za-z0-2]+@[\w]+.[\w]+')
    text = email.sub(r'',text)

    # Remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)

    # Remove longation "EX : سلاممممم = سلامم"
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)

    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])

    # remove any leading and trailing whitespace characters
    text = text.strip()
    return text

df['cleaned_text'] = df['content'].apply(clean_str)

In [5]:
def tokenize(text):
    return word_tokenize(text)

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('arabic'))
    return [word for word in tokens if word not in stop_words]

def lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in tokens]

In [6]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [7]:
df['tokens'] = df['cleaned_text'].apply(tokenize)
df['tokens'] = df['tokens'].apply(remove_stopwords)
df['tokens'] = df['tokens'].apply(lemmatize)

In [8]:
df['tokens']

0      [المحتوي, سنقدم, نظره, عامه, علي, الثقافه, الب...
1      [المحتوي, سنقدم, نظره, عامه, علي, الثقافه, الب...
2      [يستكشف, المحتوي, دور, الذكاء, الاصطناعي, مستق...
3      [يتناول, المحتوي, تحليلا, لتطور, صناعه, الالعا...
4      [يقدم, المحتوي, استعراضا, لتاثير, التغيرات, ال...
                             ...                        
223    [حاول, الغرب, علي, مدي, السنوات, القليله, الما...
224    [يحذر, خبراء, جنوب, افريقيا, عوده, الرئيس, الس...
225    [سياق, اقليمي, معقد, تمر, منطقه, افريقيا, جنوب...
226    [قرر, يوفنتوس, الايطالي, الطعن, حكم, كريستيانو...
227    [التحليل, سنستكشف, احدث, التطورات, مجال, تكنول...
Name: tokens, Length: 228, dtype: object

### 3 ) Trainning our models using RNN, Bidirectional RNN, GRU and LSTM Architectures and

In [42]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['tokens'])
X = tokenizer.texts_to_sequences(df['tokens'])
X = pad_sequences(X)
y=df['rating']
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.3, random_state=5760)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=9541)

#5760 9541


In [43]:
# Define model architectures
def build_rnn_model(input_shape):
    model = Sequential([
        Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_shape=input_shape),
        LSTM(64),
        Dense(1)
    ])
    return model

def build_bidirectional_rnn_model(input_shape):
    model = Sequential([
        Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_shape=input_shape),
        Bidirectional(LSTM(64)),
        Dense(1)
    ])
    return model

def build_gru_model(input_shape):
    model = Sequential([
        Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_shape=input_shape),
        GRU(64),
        Dense(1)
    ])
    return model

def build_lstm_model(input_shape):
    model = Sequential([
        Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_shape=input_shape),
        LSTM(64),
        Dense(1)
    ])
    return model

# Train the models
def train_model(model, X_train, y_train, X_val, y_val, epochs=10, batch_size=64):
    model.compile(optimizer='adam', loss='mean_squared_error')
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, verbose=1)
    return history

In [44]:
# Build and train RNN model
rnn_model = build_rnn_model(X_train.shape[1:])
rnn_history = train_model(rnn_model, X_train, y_train, X_val, y_val)

# Build and train Bidirectional RNN model
bidirectional_rnn_model = build_bidirectional_rnn_model(X_train.shape[1:])
bidirectional_rnn_history = train_model(bidirectional_rnn_model, X_train, y_train, X_val, y_val)

# Build and train GRU model
gru_model = build_gru_model(X_train.shape[1:])
gru_history = train_model(gru_model, X_train, y_train, X_val, y_val)

# Build and train LSTM model
lstm_model = build_lstm_model(X_train.shape[1:])
lstm_history = train_model(lstm_model, X_train, y_train, X_val, y_val)

# Evaluate the models
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return mse

rnn_mse = evaluate_model(rnn_model, X_test, y_test)
print("RNN Model MSE:", rnn_mse)

bidirectional_rnn_mse = evaluate_model(bidirectional_rnn_model, X_test, y_test)
print("Bidirectional RNN Model MSE:", bidirectional_rnn_mse)

gru_mse = evaluate_model(gru_model, X_test, y_test)
print("GRU Model MSE:", gru_mse)

lstm_mse = evaluate_model(lstm_model, X_test, y_test)
print("LSTM Model MSE:", lstm_mse)


Epoch 1/10
3/3 [==============================] - 9s 2s/step - loss: 54.2612 - val_loss: 53.7523
Epoch 2/10
3/3 [==============================] - 5s 2s/step - loss: 53.3151 - val_loss: 52.7528
Epoch 3/10
3/3 [==============================] - 6s 1s/step - loss: 51.8224 - val_loss: 50.8300
Epoch 4/10
3/3 [==============================] - 5s 1s/step - loss: 48.7803 - val_loss: 45.5660
Epoch 5/10
3/3 [==============================] - 7s 2s/step - loss: 38.8260 - val_loss: 24.5579
Epoch 6/10
3/3 [==============================] - 5s 1s/step - loss: 19.9025 - val_loss: 13.3155
Epoch 7/10
3/3 [==============================] - 5s 1s/step - loss: 11.5656 - val_loss: 8.9471
Epoch 8/10
3/3 [==============================] - 7s 2s/step - loss: 8.3146 - val_loss: 7.1114
Epoch 9/10
3/3 [==============================] - 5s 1s/step - loss: 7.0515 - val_loss: 6.4083
Epoch 10/10
3/3 [==============================] - 6s 2s/step - loss: 6.5486 - val_loss: 6.2803
Epoch 1/10
3/3 [====================